# Model Validation

By default, Django does not use Model Validation (unless we use `ModelForm`).

See: https://docs.djangoproject.com/en/2.1/ref/models/instances/#validating-objects

In [1]:
from datetime import date
from blog.models import Post

In [2]:
# cleaup section below may be helpful!
training_post = Post.objects.create(
    title='Django Training',
    slug='django-training',
    pub_date=date(2013, 1, 18),
    text=(
        "Learn Django in a classroom setting "
        "with JamBon Software."),
)
training_post

<Post: Django Training on 2013-01-18>

In [3]:
# uh-oh, we now have two Posts with the same URI!
conflict = Post.objects.create(
    title='Conflict',
    slug=training_post.slug,
    text="This object will cause problems.",
    pub_date=training_post.pub_date
)
# what happens if you try to use get_or_create above?

In [4]:
Post.objects.all()

<QuerySet [<Post: Conflict on 2013-01-18>, <Post: Django Training on 2013-01-18>]>

In [5]:
# this exception will be displayed at http://127.0.0.1:8000/blog/2013/1/django-training/
try:
    Post.objects.get(
        pub_date__year=2013,
        pub_date__month=1,
        slug='django-training'
    )
except Post.MultipleObjectsReturned as e:
    print(str(e))

get() returned more than one Post -- it returned 2!


In [6]:
conflict2 = Post(
    title='Conflict 2: The Return',
    slug=training_post.slug,
    text='More Problem Behavior in Theaters Soon!',
    pub_date=training_post.pub_date
)

In [7]:
conflict2

<Post: Conflict 2: The Return on 2013-01-18>

In [8]:
from django.core.exceptions import ValidationError
try:
    conflict2.full_clean()
except ValidationError as err:
    print(str(err))

{'slug': ['Slug must be unique for Date published month.']}


# Cleanup

In [9]:
Post.objects.filter(
    pub_date__year=2013,
    pub_date__month=1,
    slug='django-training'
).delete()

(2, {'blog.Post_tags': 0, 'blog.Post_startups': 0, 'blog.Post': 2})